In [1]:
# importing the libraries

import pandas as pd
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
from torchsummary import summary

In [2]:
None

In [3]:
#Ref: https://www.analyticsvidhya.com/blog/2019/12/image-augmentation-deep-learning-pytorch/

In [4]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, MaxPool2D, AveragePooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import matplotlib.pyplot as plt
from PIL import Image

from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, BatchNormalization, Conv2D, MaxPool2D, Activation, MaxPooling2D, Dropout,AveragePooling2D, Flatten
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import os

In [5]:
# loading dataset
data = pd.read_csv('lowhead_train_equal.csv')
data.head()
print(data)

            image_name  lowhead_or_not
0            7-118.png               1
1              8-4.png               1
2              9-1.png               1
3              9-2.png               1
4              9-4.png               1
..                 ...             ...
319  sample_168000.png               0
320  sample_169000.png               0
321  sample_170000.png               0
322  sample_171000.png               0
323  sample_172000.png               0

[324 rows x 2 columns]


In [6]:
# loading images
import glob
lowheads_vs_other = glob.glob('lowheadVsother/*.*')

train_img = []
for img_name in tqdm(data['image_name']):
    img_name = 'lowheadVsother/'+img_name
    for i in lowheads_vs_other:
        if(i == img_name):
            image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb')
            image=np.array(image)
            image = image/255
            train_img.append(image)

train_x = np.array(train_img)
train_y = data['lowhead_or_not'].values
train_x.shape, train_y.shape





100%|██████████| 324/324 [00:11<00:00, 27.96it/s]


((324, 500, 500, 3), (324,))

In [10]:
final_train_data = []
final_train_label = []
for i in tqdm(range(train_x.shape[0])):
    final_train_data.append(train_x[i])
    final_train_data.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_data.append(np.fliplr(train_x[i]))
    final_train_data.append(np.flipud(train_x[i]))
    final_train_data.append(random_noise(train_x[i],var=0.2**2))
    for j in range(5):
        final_train_label.append(train_y[i])

100%|██████████| 259/259 [00:24<00:00, 10.51it/s]


In [12]:
len(final_train_label), len(final_train_data)
final_train = np.array(final_train_data)
final_train_label = np.array(final_train_label)
print(len(final_train_label))
print(len(final_train_data))

1295
1295


In [14]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [15]:
# converting training images into torch format
final_train = final_train.reshape(1295, 3, 500, 500)
final_train  = torch.from_numpy(np.asarray(final_train))
final_train = final_train.float()

# converting the target into torch format
final_train_label = final_train_label.astype(int)
final_train_label = torch.from_numpy(final_train_label)

In [16]:
# converting validation images into torch format
val_x = val_x.reshape(65, 3, 500, 500)
val_x  = torch.from_numpy(val_x)
val_x = val_x.float()

# converting the target into torch format
val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

In [17]:
torch.manual_seed(0)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
            # Defining another 2D convolution layer
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            # adding batch normalization
            BatchNorm2d(128),
            MaxPool2d(kernel_size=2, stride=2),
            # adding dropout
            Dropout(p=0.25),
        )

        self.linear_layers = Sequential(
            Linear(128 * 31 * 31, 512), #25088
            ReLU(inplace=True),
            Dropout(),
            Linear(512, 256),
            ReLU(inplace=True),
            Dropout(),
            Linear(256,10),
            ReLU(inplace=True),
            Dropout(),
            Linear(10,2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [18]:
## 
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.000075)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available


device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if torch.cuda.is_available():
    model.to(torch.device('cpu:0'))
    criterion = criterion.cpu()

print(model)

Using device: cpu

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.25, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.25, inplace=F

In [19]:
#Training the Model

torch.manual_seed(0)

# batch size of the model
batch_size = 64

# number of epochs to train the model
n_epochs = 10

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
        
    permutation = torch.randperm(final_train.size()[0])

    training_loss = []
    for i in tqdm(range(0,final_train.size()[0], batch_size)):

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = final_train[indices], final_train_label[indices]
        
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cpu(), batch_y.cpu()
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs,batch_y)

        training_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        
    training_loss = np.average(training_loss)
    print('epoch: \t', epoch, '\t training loss: \t', training_loss)
    

  0%|          | 0/21 [00:00<?, ?it/s]

epoch: 	 1 	 training loss: 	 0.6998126819020226


  0%|          | 0/21 [00:00<?, ?it/s]

epoch: 	 2 	 training loss: 	 0.5828504746868497


  0%|          | 0/21 [00:00<?, ?it/s]

...

epoch: 	 8 	 training loss: 	 0.4053116483347757


  0%|          | 0/21 [00:00<?, ?it/s]

epoch: 	 9 	 training loss: 	 0.3855597291673933


100%|██████████| 21/21 [04:58<00:00, 14.20s/it]

epoch: 	 10 	 training loss: 	 0.38759595794337137


In [20]:
torch.save(model, 'model.pt')

In [21]:
CNN_model = torch.load('model.pt')

In [22]:
torch.manual_seed(0)
# prediction for training set
predict = []
target = []
permutation_train = torch.randperm(final_train.size()[0])
for i in tqdm(range(0,final_train.size()[0], batch_size)):
    index = permutation_train[i:i+batch_size]
    batch_x, batch_y = final_train[index], final_train_label[index]

    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cpu(), batch_y.cpu()

    with torch.no_grad():
        output = model(batch_x.cpu())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    prediction = np.argmax(prob, axis=1)
    predict.append(prediction)
    target.append(batch_y)
    
# training accuracy
accuracy = []
for i in range(len(predict)):
    accuracy.append(accuracy_score(target[i].cpu(),predict[i]))
    
print('training accuracy: \t', np.average(accuracy))


100%|██████████| 21/21 [02:01<00:00,  5.77s/it]

training accuracy: 	 0.836656746031746


In [23]:
# checking the performance on validation set
torch.manual_seed(0)
output = model(val_x.cpu())
softmax = torch.exp(output).cpu()
prob = list(softmax.detach().numpy())
predict = np.argmax(prob, axis=1)
accuracy_score(val_y, predict)

0.8153846153846154

In [27]:
#print(accuracy_score(target[i].cpu(),prediction[i]))
v=accuracy_score(val_y, predict)
for i, j in zip(val_y, predict):
    print(str(i) + " / " + str(j))

tensor(0) / 0
tensor(1) / 1
tensor(0) / 1
tensor(0) / 0
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(0) / 0
tensor(1) / 0
tensor(0) / 0
tensor(0) / 0
tensor(1) / 1
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(1) / 1
tensor(0) / 0
tensor(1) / 0
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(0) / 0
tensor(0) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 1
tensor(0) / 0
tensor(1) / 1
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(0) / 0
tensor(0) / 0
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(1) / 1
tensor(1) / 1
tensor(1) / 0
tensor(0) / 1
tensor(1) / 0
tensor(1) / 1
tensor(1) / 1
tensor(1) / 1
tensor(0) / 0
tensor(1) / 1
tensor(0) / 0
tensor(0) / 1
tensor(1) / 0
tensor(0) / 0
tensor(1) / 1
tensor(0) / 1
tensor(0) / 1
tensor(0) / 0
tensor(1) / 1
tensor(1) / 1
tensor(1) / 1
tensor(1) / 1
